<a href="https://colab.research.google.com/github/YiyoMb/extraccion-conocimiento-bd/blob/main/notebooks/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# PROYECTO: EXTRACCIÓN DE CONOCIMIENTO EN BASES DE DATOS
# Configuración inicial del entorno de trabajo

# ==============================================================================
# PASO 1: INSTALACIÓN DE LIBRERÍAS NECESARIAS
# ==============================================================================

# Instalamos las librerías que no vienen por defecto en Colab
!pip install plotly dash
!pip install mlxtend  # Para análisis de asociación (Market Basket Analysis)
!pip install wordcloud  # Para visualizaciones avanzadas

# ==============================================================================
# PASO 2: IMPORTACIÓN DE LIBRERÍAS PRINCIPALES
# ==============================================================================

# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning - Modelos Supervisados
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report

# Machine Learning - Modelos No Supervisados
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Análisis de Asociación
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Utilidades
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Todas las librerías importadas correctamente")
print("✅ Entorno configurado para el proyecto")

# ==============================================================================
# PASO 3: CONFIGURACIÓN DE PARÁMETROS GLOBALES
# ==============================================================================

# Configuración para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configuración de visualizaciones
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Configuración global aplicada")
print("🚀 ¡Listo para comenzar el análisis de datos!")

# ==============================================================================
# DESCARGA Y PREPARACIÓN DEL DATASET: ONLINE RETAIL
# ==============================================================================

# Descargar el dataset desde UCI
import urllib.request
import os

print("📥 Descargando dataset Online Retail...")

# URL del dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
filename = "Online_Retail.xlsx"

# Crear carpeta de datos si no existe
if not os.path.exists('datos'):
    os.makedirs('datos')

# Descargar el archivo
try:
    urllib.request.urlretrieve(url, f'datos/{filename}')
    print("✅ Dataset descargado exitosamente")
except Exception as e:
    print(f"❌ Error al descargar: {e}")
    print("💡 Descarga manual desde: https://archive.ics.uci.edu/ml/datasets/Online+Retail")

# ==============================================================================
# CARGA INICIAL DEL DATASET
# ==============================================================================

try:
    # Cargar el dataset
    print("\n📊 Cargando dataset...")
    df = pd.read_excel(f'datos/{filename}')

    print("✅ Dataset cargado exitosamente")
    print(f"📏 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")

    # Información básica del dataset
    print("\n🔍 INFORMACIÓN BÁSICA DEL DATASET:")
    print("="*50)
    print(f"📋 Columnas: {list(df.columns)}")
    print(f"💾 Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Primeras filas
    print("\n👀 PRIMERAS 5 FILAS:")
    print("="*50)
    print(df.head())

    # Información de tipos de datos
    print("\n📊 TIPOS DE DATOS:")
    print("="*50)
    print(df.info())

    # Estadísticas básicas
    print("\n📈 ESTADÍSTICAS BÁSICAS:")
    print("="*50)
    print(df.describe())

except FileNotFoundError:
    print("❌ Archivo no encontrado. Por favor, descarga manualmente el dataset.")
    print("📁 Guárdalo en la carpeta 'datos' con el nombre 'Online_Retail.xlsx'")
except Exception as e:
    print(f"❌ Error al cargar el dataset: {e}")

print("\n🎯 SIGUIENTE PASO: Análisis exploratorio de datos")
print("📝 Crearemos el notebook '01_analisis_exploratorio.ipynb'")

# ==============================================================================
# CONFIGURACIÓN GIT PARA COLAB - EJECUTAR SOLO UNA VEZ
# ==============================================================================

# Configura tu identidad (CAMBIAR POR TUS DATOS REALES)
!git config --global user.email "diegobalderas.864eyou@gmail.com"
!git config --global user.name "YiyoMb"

print("✅ Git configurado correctamente")
email = !git config --global user.email
print("📧 Email:", email[0])
name = !git config --global user.name
print("👤 Nombre:", name[0])

# Verificar que GitHub funciona (opcional)
print("\n🔗 Para verificar conexión con GitHub:")
print("   1. Ve a Archivo > Guardar copia en GitHub")
print("   2. Selecciona tu repositorio")
print("   3. Si aparece, ¡todo está funcionando!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.0 MB/s eta 0:00:00
✅ Todas las librerías importadas correctamente
✅ Entorno configurado para el proyecto
✅ Configuración global aplicada
🚀 ¡Listo para comenzar el análisis de datos!
📥 Descargando dataset Online Retail...
✅ Dataset descargado exitosamente

📊 Cargando dataset...
✅ Dataset cargado exitosamente
📏 Dimensiones: 541,909 filas × 8 columnas

🔍 INFORMACIÓN BÁSICA DEL DATASET:
📋 Columnas: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
💾 Memoria utilizada: 134.93 MB

👀 PRIMERAS 5 FILAS:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E    